In [2]:
from langchain.llms import Bedrock
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import boto3


def get_bedrock_client():
    bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")
    return bedrock_client

# Instantiate Bedrock LLM
llm = Bedrock(
    client=get_bedrock_client(),
    model_id="amazon.anthropic.claude-v2"
)

conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:


ValueError: Error raised by bedrock service: Unable to locate credentials